In [3]:
import json
import sys
from tqdm import tqdm
sys.path.append('..')
from grammar.eval.match import SemanticsMatch


def judge_retrieval_result(result):
    result['true_document_ids'] = set([int(r) for r in result['true_document_ids']])
    result['retrieved_document_ids'] = set([int(r) for r in result['retrieved_document_ids']])
    result['retrieval_judgement'] = 1 if result['true_document_ids'] == result['retrieved_document_ids'] else 0
    return result

def judge_rag_result(result, semnatics_match, open_domain=False):
    if 'judgement' in result:
        print("Judgement already exists")
        return result
    if open_domain and result['retrieval_judgement'] == 0:
        result["judgement"] = "Incorrect"
    else:
        result["judgement"] = semnatics_match.generate((result["query"], result["answer"], result["gpt_response"][0]), verbose=True)[0]
    return result


linguistic_attr = 'long'
root_dir = 'spider'
open_domain = True
semnatics_match = SemanticsMatch.from_file(root_dir=root_dir, verbalize_attrs=linguistic_attr)
file_path = f'{root_dir}/eval_results/results_{linguistic_attr}_balanced.json'
with open(file_path) as f:
    results = json.load(f)
num_retrieval_failure = sum([result['retrieval_judgement']==0 for result in results])
print(f"Retrieval failed in {num_retrieval_failure} out of {len(results)} examples")
import time
for i, result in tqdm(enumerate(results)):
    # sleep for 20 seconds after 9 examples
    # if results.index(result) % 9 == 0 and results.index(result) != 0:
    #     print("Sleeping for 20 seconds")
    #     time.sleep(20)
    #     print("Waking up")
    result = judge_rag_result(result, semnatics_match, open_domain)
num_rag_failure = sum([result['judgement']=="Incorrect" for result in results])
print(f"RAG failed in {num_rag_failure} out of {len(results)} examples")



Retrieval failed in 412 out of 570 examples


0it [00:00, ?it/s]

The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
The 1 generations for the input `k` exist in `cache_generations`! No need to gen

381it [00:08, 44.09it/s]

LLM is generating... for the key:  ("We are trying to determine the nationality of a person identified as 'Reggie Lewis' in the 'people' database. Can you provide that information?", 'United States', "I'm sorry, but I don't have access to specific databases or personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. My primary function is to provide information and answer questions to the best of my knowledge and abilities. If you have any concerns about privacy or data security, please let me know, and I will do my best to address them.")


...

LLM is generating... for the key:  ("Our target is to pinpoint the graduate school related to the person who goes by the name 'David Butler'.", 'California', "I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I don't have the ability to pinpoint the graduate school related to a specific person like 'David Butler' unless that information has been provided to me. My primary function is to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I can help you with?")


503it [01:10,  1.33it/s]

LLM is generating... for the key:  ("We are attempting to identify the graduation college associated with the person whose name is given as 'Tim Naegeli'.", 'Wisconsin–Stevens Point', "I'm sorry, but I couldn't find any information about the graduation college associated with a person named 'Tim Naegeli'. It's possible that this information is not publicly available or that there is no notable person with that name in relation to a specific graduation college.")


570it [01:11,  8.00it/s]

The 1 generations for the input `k` exist in `cache_generations`! No need to generate more.
RAG failed in 448 out of 570 examples


In [4]:
semnatics_match.save(root_dir=f'{root_dir}', override=True)
semnatics_match.llm.gpt_usage_record.write_usage(model_name='chatgptk' )

In [5]:
# ensure json serializable
for result in results:
    result['true_document_ids'] = list(result['true_document_ids'])
    result['retrieved_document_ids'] = list(result['retrieved_document_ids'])

# save results
with open(f'{root_dir}/eval_results/results_{linguistic_attr}_balanced.json', 'w') as f:
    json.dump(results, f, indent=4)